In [1]:
import os
import math

import chromadb
import cv2
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from mysql.connector import connect
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tqdm.notebook import tqdm

In [2]:
load_dotenv()

True

In [3]:
mysql = connect(host="localhost", user="root", password="root", database="lecture_5_demo")

In [4]:
def create_database():
  with mysql.cursor() as cursor:
    cursor.execute("CREATE DATABASE lecture_5_demo")

In [5]:
# create_database()

In [6]:
def create_table():
  query = """
    CREATE TABLE crops (
      id INT PRIMARY KEY AUTO_INCREMENT,
      country_or_area LONGTEXT,
      year LONGTEXT,
      element LONGTEXT,
      element_code LONGTEXT,
      category LONGTEXT,
      unit LONGTEXT,
      value DOUBLE,
      value_footnotes LONGTEXT
    )
  """

  with mysql.cursor() as cursor:
    cursor.execute(query)

In [7]:
# create_table()

In [8]:
storage_options = {
    "key": os.environ.get("S3_KEY"),
    "secret": os.environ.get("S3_SECRET"),
    "client_kwargs": {"endpoint_url": "http://blizzard.inside.yjoer.com:9101"},
}

df_crops = pd.read_csv(
    "s3://datasets/global-food-agriculture-statistics/fao_data_crops_data.csv",
    storage_options=storage_options,
)

df_crops.replace(np.nan, None, inplace=True)

In [9]:
def insert_rows(rows):
  query = """
    INSERT INTO crops (
      country_or_area, year, element, element_code,
      category, unit, value, value_footnotes
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
  """

  records = []

  for idx, row in rows:
    records.append((
      row.country_or_area, row.year, row.element, row.element_code,
      row.category, row.unit, row.value, row.value_footnotes
    ))

  with mysql.cursor() as cursor:
    cursor.executemany(query, records)
    mysql.commit()

In [10]:
def insert_data():
  batch_size = 1000
  batches = np.array_split(df_crops, len(df_crops) // batch_size + 1)

  for batch in tqdm(batches):
    insert_rows(batch.iterrows())

In [11]:
# insert_data()

In [12]:
def get_area_harvested_in_malaysia():
  query = """
    SELECT year, SUM(value)
    FROM crops
    WHERE country_or_area = 'Malaysia' AND element = 'Area Harvested'
    GROUP BY year
    ORDER BY year
  """

  with mysql.cursor() as cursor:
    cursor.execute(query)

    for x in cursor:
      print(x)

In [13]:
get_area_harvested_in_malaysia()

('1961.0', 3971026.0)
('1962.0', 4073968.0)
('1963.0', 4321595.0)
('1964.0', 4381823.0)
('1965.0', 4594246.0)
('1966.0', 4499497.0)
('1967.0', 4599414.0)
('1968.0', 4954188.0)
('1969.0', 5184958.0)
('1970.0', 5404319.0)
('1971.0', 5636415.0)
('1972.0', 5954584.0)
('1973.0', 6337340.0)
('1974.0', 6461321.0)
('1975.0', 6621428.0)
('1976.0', 6898096.0)
('1977.0', 7200411.0)
('1978.0', 7141118.0)
('1979.0', 7579654.0)
('1980.0', 7853297.0)
('1981.0', 8114167.0)
('1982.0', 8114858.0)
('1983.0', 8409470.0)
('1984.0', 8389765.0)
('1985.0', 8911615.0)
('1986.0', 9317489.0)
('1987.0', 9538506.0)
('1988.0', 10209272.0)
('1989.0', 10599297.0)
('1990.0', 10881397.0)
('1991.0', 11238651.0)
('1992.0', 11284711.0)
('1993.0', 11639990.0)
('1994.0', 11936744.0)
('1995.0', 12006889.0)
('1996.0', 12334110.0)
('1997.0', 12575656.0)
('1998.0', 12891011.0)
('1999.0', 13434915.0)
('2000.0', 14196342.0)
('2001.0', 14700691.0)
('2002.0', 14913356.0)
('2003.0', 14626540.0)
('2004.0', 15032362.0)
('2005.0', 1550

In [14]:
images = [
  "L4/apple.jpg",
  "L4/orange.jpg",
  "L4/banana.jpg",
  "L4/honeydew.png",
  "L4/melon.png",
  "L4/banana_single.jpg"
]

names = ["apple", "orange", "banana", "honeydew", "melon", "banana_query"]

In [15]:
ims = []

for image in images:
  im = cv2.imread(image)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

  ims.append(im)

In [16]:
processed_ims = []

for im in ims:
  scaled_im = cv2.resize(im, (224, 224))
  processed_im = preprocess_input(scaled_im)

  processed_ims.append(processed_im)

processed_ims = np.array(processed_ims)

In [17]:
base_model = ResNet50(include_top=False)
pooling = GlobalAveragePooling2D()(base_model.output)

model = Model(inputs=base_model.input, outputs=pooling)

In [18]:
embeddings = model.predict(processed_ims)
embeddings.shape

1/1 [==============================] - 3s 3s/step


(6, 2048)

In [19]:
query = embeddings[5]
embeddings_index = embeddings[0:5]

similarity = cosine_similarity(np.expand_dims(query, axis=0), embeddings_index)

sorted(list(zip(names, similarity[0])), key=lambda x: x[1], reverse=True)

[('banana', 0.6324712),
 ('honeydew', 0.5105934),
 ('melon', 0.4509704),
 ('orange', 0.35673666),
 ('apple', 0.3533535)]

In [20]:
client = chromadb.PersistentClient(path="./data/chroma")

collection = client.get_or_create_collection(
  name="lecture_5_demo",
  metadata={"hnsw:space": "cosine"}
)

In [21]:
def insert_embeddings():
  for idx, embedding in enumerate(embeddings_index):
    collection.add(embeddings=[embedding.tolist()], ids=[names[idx]])

In [22]:
# insert_embeddings()

In [23]:
results = collection.query(query_embeddings=[query.tolist()], n_results=5)

list(zip(results['ids'][0], results['distances'][0]))

[('banana', 0.3675287573361994),
 ('honeydew', 0.4894066374567593),
 ('melon', 0.5490295951677537),
 ('orange', 0.6432633620945717),
 ('apple', 0.6466465706627192)]